<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/Clock_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.integrate as integrate
 
 # degradation rates for nuclear proteins and nuclear protein complexes
    
dx1=0.08  # degradation rate for CLOCK/BMAL
dx2=0.06  # degradation rate for PER*/CRYn
dx3=0.09  # degradation rate for PER/CRYn
dx5=0.17  # degradation rate for REV-ERB
dx6=0.12  # degradation rate for RORn
dx7=0.15  # degradation rate for BMALn
 

 # degradation rates for mRNAs 
   
dy1=0.3   # degradation rate for Per
dy2=0.2   # degradation rate for Cry
dy3=2     # degradation rate for Rev-Erb
dy4=0.2   # degradation rate for Ror
dy5=1.6   # degradation rate for Bmal


  # degradation rates for cytoplasmic proteins and cytoplasmic protein complexes 
  
dz1=0.23   # degradation rate for CRY 
dz2=0.25   # degradation rate for PER 
dz3=0.6    # degradation rate for PER*
dz4=0.2    # degradation rate for PER*/CRYc
dz5=0.2    # degradation rate for PER/CRYc
dz6=0.31   # degradation rate for REV-ERB
dz7=0.3    # degradation rate for RORc
dz8=0.73   # degradation rate for BMALc 


  # reaction rates for complex formation/dissociation
  
kfx1=2.3   # complex formation rate for CLOCK/BMAL
kdx1=0.01  # complex dissociation rate for CLOCK/BMAL

kfz4=1     # complex formation rate for PER*/CRY
kfz5=1     # complex formation rate for PER/CRY

kdz5=1     # complex dissociation rate for PER/CRY
kdz4=1     # complex dissociation rate for PER*/CRY


  # phosphorylation rates 

kfz2=2      # phosphorylation rate constant for PER  
kdfz3=0.05  # dephosphorylation rate constant for PER*


  # transcription rates
  
V1max=1      # maximal rate of Per transcription
V2max=2.92   # maximal rate of Cry transcription
V3max=1.90   # maximal rate of Rev-Erb transcription
V4max=10.90  # maximal rate of Ror transcription
V5max=1      # maximal rate of Bmal transcription

kt1=3     # rate constant of Per transcription
kt2=2.4   # rate constant of Cry transcription
kt3=2.07  # rate constant of Rev-Erb transcription
kt4=0.9   # rate constant of Ror transcription
kt5=8.35  # rate constant of Bmal transcription

ki1=0.9   # inhibition constant of Per transcription
ki2=0.7   # inhibition constant of Cry transcription
ki21=5.2  # inhibition constant of Cry transcription by REV-ERB
ki3=3.3   # inhibition constant of Rev-Erb transcription
ki4=0.4   # inhibition constant of Ror transcription
ki5=1.94  # inhibition constant of Bmal transcription


   # translation rates
    
kp1=0.4     # production rate of PER  
kp2=0.26    # production rate of CRY
kp3=0.37    # production rate of REV-ERB 
kp4=0.76    # production rate of ROR
kp5=1.21    # production rate of BMAL

   # import/export rates
    
kiz4=0.2   # import rate of  PER*/CRY to the nucleus
kiz5=0.1   # import rate of  PER/CRY to the nucleus
kiz6=0.5   # import rate of REV-ERB to the nucleus
kiz7=0.1   # import rate of ROR to the nucleus
kiz8=0.1   # import rate of BMAL to the nucleus

kex2=0.02  # export rate of  PER*/CRY to the cytoplasm
kex3=0.01  # export rate of  PER/CRY to the cytoplasm


   # fold activation of transcription
 
a=12  # fold activation of Per transcription
d=12  # fold activation of Cry transcription
g=5   # fold activation of Rev-Erb transcription 
h=5   # fold activation of Ror transcription
i=12  # fold activation of Bmal transcription


  # Hill coefficients of transcription

b=5    # Hill coefficient of activation of Per transcription
c=7    # Hill coefficient of inhibition of Per transcription

e=6    # Hill coefficient of activation of Cry transcription
f=4    # Hill coefficient of inhibition of Cry transcription
f1=1   # Hill coefficient of inhibition of Cry transcription via REV

v=6    # Hill coefficient of activation of Rev-Erb  transcription
w=2    # Hill coefficient of inhibition of Rev-Erb  transcription

p=6    # Hill coefficient of activation of Ror transcription
q=3    # Hill coefficient of inhibition of Ror transcription

n=2    # Hill coefficient of activation of Bmal transcription
m=5    # Hill coefficient of inhibition of Bmal transcription


a2=1  # weight factor for PER*/CRY
a3=1  # weight factor for PER/CRY

Rev0=0   # simulating a constitutively expressed Rev-Erb with the given value
Ror0=0   # simulating a constitutively expressed Ror with the given value
Bmal0=0  # simulating a constitutively expressed Bmal with the given value
Per0=0   # simulating a constitutively expressed Per with the given value
Cry0=0   # simulating a constitutively expressed Cry with the given value

 
dt = 0.01
t = np.arange(0.0, 50.0, dt) #The time to integrate over 
len_t = len(t) 
timestep = 20 #timestep
 
timecount = 0 #global

In [9]:
#ODEs

def PC(x18, x19):
    return a2 * np.array(x18) + a3 * np.array(x19) # pool of nuclear complexes PER/CRY

def CLOCK_BMAL(x1, x10):
    return (kfx1 * np.array(x10) - kdx1 * np.array(x1) - dx1 * np.array(x1))   # CLOCK/BMAL

def Rev_erb_m(x1, PC, x2):
    return (V3max *  np.array(1 + g *  np.array((x1 / kt3)**v)) / (1 + ((PC / ki3)**w) *  np.array((x1 / kt3)**v) + (x1 / kt3)**v ) - dy3 *  np.array(x2))   # transcription of Rev-erb

def Ror_m(x1, PC, x3):
    return (V4max *  np.array(1 + h *  np.array((x1 / kt4)**p)) / (1 + ((PC / ki4) **q) *  np.array((x1 / kt4) **p) + (x1 / kt4)  **p ) - dy4 *  np.array(x3));   # transcription of Ror

def REV_ERB_p(x2, x4):
    return (kp3 * np.array(x2) - kiz6 *  np.array(x4() - dz6 *  np.array(x4))   # production of REV-ERB 

def ROR_p(x3, x5):
    return (kp4 * np.array(x3) - kiz7 * np.array(x5) - dz7 * np.array(x5)) # production of ROR

def REV_ERB_p_n(x4, x6):
    return (kiz6 *  np.array(x4) - dx5 *  np.array(x6))   # nuclear REV-ERB 

def ROR_p_n(x5, x7):
    return (kiz7 *  np.array(x5) - dx6 *  np.array(x7))   # nuclear ROR

def Bmal_m(x6, x7, x8):
    return (V5max *  np.array(1 + i *  np.array((x7 / kt5) ** n)) / (1 + (x6 / ki5) ** m + (x7 / kt5) ** n ) - dy5 *  np.array(x8)) # transcription of Bmal

def BMAL_p(x8, x9):
    return (kp5 *  np.array(x8 + Bmal0) - kiz8 *  np.array(x9) - dz8 *  np.array(x9))   # production of BMAL

def BMAL_p_n(x1, x9, x10):
    return (kiz8 *  np.array(x9) + kdx1 *  np.array(x1) -  kfx1 *  np.array(x10) - dx7 *  np.array(x10))   # nuclear BMAL

def Per_m(x1, PC, x11):
    return (V1max *  np.array(1 + a *  np.array((x1 / kt1) ** b)) / (1 + ((PC / ki1) ** c) *  np.array((x1 / kt1 ) ** b)  + (x1 / kt1 ) ** b) - dy1 *  np.array(x11)) # transcription of Per

def Cry_m(x1, PC, x6, x12):
    return ((V2max *  np.array(1 + d *  np.array((x1 / kt2) ** e)) / (1 + ((PC / ki2) ** f) * np.array((x1 / kt2 ) ** e)  + (x1 / kt2 ) ** e)) *  np.array(1 / (1 + (x6 / ki21) ** f1)) - dy2 * np.array(x12)) # transcription of Cry

def CRY_p(x12, x13,x14,x15,x16,x17):
    return (kp2 * np.array(x12)  + kdz4 * np.array(x16) + kdz5 * np.array(x17) - kfz5 * np.array(x13) * np.array(x14) - kfz4 * np.array(x13) * np.array(x15) - dz1 * np.array(x13)) # production of CRY 

def PER_p(x11, x13,x14,x15,x17):
    return (kp1 * np.array(x11)  + kdz5 * np.array(x17)  + kdfz3 * np.array(x15) - kfz5 * np.array(x14) * np.array(x13) - kfz2 * np.array(x14) - dz2 * np.array(x14)) # production of PER 

def PER_p_p(x13,x14,x15,x16):
    return (kfz2 * np.array(x14) + kdz4 * np.array(x16) - kdfz3 * np.array(x15) - kfz4 * np.array(x15) * np.array(x13)  - dz3 * np.array(x15)) # phosphorylation of PER 

def PER_CRY_com_c_p(x13,x15,x16,x18):
    return (kfz4 * np.array(x13) * np.array(x15) + kex2 * np.array(x18) - kiz4 * np.array(x16) - kdz4 * np.array(x16) - dz4 * np.array(x16))  # complex formation-PER*/CRY -cytoplasm

def PER_CRY_com_c(x13,x14,x17,x19):
    return (kfz5 * np.array(x13) * np.array(x14) + kex3 * np.array(x19) - kiz5 * np.array(x17) - kdz5 * np.array(x17) - dz5 * np.array(x17)) # complex formation-PER/CRY -cytoplasm

def PER_CRY_com_n_p(x16,x18):
    return (kiz4 * np.array(x16) - kex2 * np.array(x18) - dx2 * np.array(x18)) # nuclear complex-PER*/CRY 

def PER_CRY_com_n(x17,x19):
    return (kiz5 * np.array(x17) - kex3 * np.array(x19) - dx3 * np.array(x19)) # nuclear complex-PER/CRY

def dALLdt(X, t):

   global timecount
   
   PC, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19 = X

   PC_ =  PC(x18, x19) # pool of nuclear complexes PER/CRY

   x1_ = CLOCK_BMAL(x1, x10)  # CLOCK/BMAL

   x2_ = Rev_erb_m(x1, PC, x2)   # transcription of Rev-erb

   x3_ = Ror_m(x1, PC, x3)   # transcription of Ror

   x4_ = REV_ERB_p(x2, x4)   # production of REV-ERB 

   x5_ = ROR_p(x3, x5)   # production of ROR

   x6_ = REV_ERB_p_n(x4, x6)   # nuclear REV-ERB 

   x7_ = ROR_p_n(x5, x7)   # nuclear ROR

   x8_ = Bmal_m(x6, x7, x8) # transcription of Bmal

   x9_ = BMAL_p(x8, x9)   # production of BMAL

   x10_ = BMAL_p_n(x1, x9, x10)   # nuclear BMAL

   x11_ = Per_m(x1, PC, x11) # transcription of Per

   x12_ = Cry_m(x1, PC, x6, x12) # transcription of Cry

   x13_ = CRY_p(x12, x13,x14,x15,x16,x17) # production of CRY 

   x14_ = PER_p(x11, x13,x14,x15,x17) # production of PER 

   x15_ = PER_p_p(x13,x14,x15,x16) # phosphorylation of PER 

   x16_ = PER_CRY_com_c_p(x13,x15,x16,x18)  # complex formation-PER*/CRY -cytoplasm

   x17_ = PER_CRY_com_c(x13,x14,x17,x19) # complex formation-PER/CRY -cytoplasm

   x18_ = PER_CRY_com_n_p(x16,x18) # nuclear complex-PER*/CRY 

   x19_ = PER_CRY_com_n(x17,x19)  # nuclear complex-PER/CRY 

   return PC_, x1_, x2_, x3_, x4_, x5_, x6_, x7_, x8_, x9_, x10_, x11_, x12_, x13_, x14_, x15_, x16_, x17_, x18_, x19_
  
   X0 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
   X= integrate.odeint(dALLdt, X0, t)


   PC = X[:,0]
  
   print(PC)


     


SyntaxError: ignored